In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import Huber
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In my previous post, I went through the following process:

Environment: Anaconda, Windows 11.

- Data wrangling (Light exploration, followed by removing and transforming some variables)
- Split the training dataset into training and validation datasets
- Fit a model using a Scikit-Learn pipeline (Data Preprocessing + fitting XGBoost/LightGBM estimators with a Randomized Search across their respective hyperparameters)
- Evaluate and visualize model performance
- Implement an automated approach to selecting hyperparameters (HyperOpt)
- Make predictions

In this post, I will implement the following using the same wrangled/preprocessed data:

Environment: Docker, Windows Subsystem for Linux 2 (WSL 2), Windows 11.

- Build a simple Sequential model in Keras/Tensorflow
- Use the Weights and Biases (WandB) platform to select optimal hyperparameters and record experiments.
  - Experiments are evaluated using K-Fold Cross Validation. Mean RMSE across folds for each experiment are custom logged in WandB. 
- Make predictions.
- Blend predictions from the previous post (decision tree) and this post (neural net).
  - By both taking the mean of predictions, and defining a meta-model trained on a holdout dataset kept completely separate.

In [32]:
# Function to bring in wrangled/preprocessed data from previous post
def data():
    training = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/training_preprocessed")
    validation = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/validation_preprocessed")
    holdout = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preprocessed")
    holdout_predictions_df = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/holdout_preds_preprocessed")
    test = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/test_preprocessed")
    
    X_train = training.drop(columns="SalePrice")
    y_train = training["SalePrice"]
    X_valid = validation.drop(columns="SalePrice")
    y_valid = validation["SalePrice"]
    X_holdout = holdout.drop(columns="Actual_SalePrice")
    y_holdout = holdout["Actual_SalePrice"]
    X_test = test
    holdout_predictions_df = holdout_predictions_df
    return X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df

# Bring in data
X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df = data()

# Since this model uses k-fold validation, we don't need separate training and validation datasets
X_train = X_train.append(X_valid).reset_index().drop(columns="index")
y_train = y_train.append(y_valid).reset_index().drop(columns="index").values

<ipython-input-32-20af1fdf8908>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_valid).reset_index().drop(columns="index")
<ipython-input-32-20af1fdf8908>:24: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_valid).reset_index().drop(columns="index").values


In [4]:
# Log into Weights and Biases
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [11]:
# Define simple Sequential model
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(wandb.config.dense1, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(wandb.config.dense2, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss=Huber(), metrics=[RootMeanSquaredError()])
    
    return model

In [13]:
# Define training function and hyperparameter ranges
from wandb.keras import WandbCallback
from sklearn.model_selection import KFold

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.3
    },
    "dense1": {
      "values": [8, 16, 32]
    },
    "dense2": {
      "values": [8, 16, 32]
    },
    "epochs": {
      "values": [250,500,750]
    },
    "batch_size": {
      "values": [8, 16, 64, 128]
    }
  },
  "metric": {
    "name": "Mean Validation RMSE (all folds)",
    "goal": "minimize"
  }
}

config_defaults = {
  "dropout1": 0.1,
  "dense1": 16,
  "dense2": 16,
  "epochs": 100,
  "batch_size": 16
}

# Define number of splits
kf = KFold(n_splits=5)

def train():

  rmse_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(X_train), 
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
      
      # Generate data for each
      scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
      rmse_per_fold.append(scores[1])
      loss_per_fold.append(scores[0])

      # Increase fold number
      fold_no = fold_no + 1

      # == Provide average scores ==
      print('------------------------------------------------------------------------')
      print('Score per fold')
      for i in range(0, len(rmse_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - RMSE: {rmse_per_fold[i]}')
      print('------------------------------------------------------------------------')
      print('Average scores for all folds:')
      print(f'> RMSE: {np.mean(rmse_per_fold)} (+- {np.std(rmse_per_fold)})')
      print(f'> Loss: {np.mean(loss_per_fold)}')
      print('------------------------------------------------------------------------')

      wandb.log({
        "Mean Validation RMSE (all folds)": np.mean(rmse_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 10

wandb.agent(keras_sequential_sweep_1, function=train, count=count)

Create sweep with ID: r64dtrnj
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/r64dtrnj


wandb: Agent Starting Run: u63rieuv with config:
wandb: 	batch_size: 64
wandb: 	dense1: 32
wandb: 	dense2: 16
wandb: 	dropout1: 0.1075711633874446
wandb: 	epochs: 250
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


37/37 [==============================] - 0s 954us/step - loss: 0.6825 - root_mean_squared_error: 1.1869
Score for fold 1: loss of 0.6825138330459595; root_mean_squared_error of 1.1868821382522583
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.6825138330459595 - RMSE: 1.1868821382522583
------------------------------------------------------------------------
Average scores for all folds:
> RMSE: 1.1868821382522583 (+- 0.0)
> Loss: 0.6825138330459595
------------------------------------------------------------------------


Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃▂▂▁▁▂▅▆▃▄▄▅▄▃▃▅▆▇▄▅▄▆▇▅▃▇▅▇█▆▅▄▆▆▅▄▅▅▄▇
val_root_mean_squared_error,▄▃▃▁▁▂▆▆▃▅▄▅▅▃▄▅▇▇▅▅▄▇▇▆▄▇▅▇█▆▅▄▆▆▆▅▆▆▅▇
Mean Validation Loss (all folds),0.68251
Mean Validation RMSE (all folds),1.18688
best_epoch,18
best_val_loss,0.02627


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

37/37 [==============================] - 0s 783us/step - loss: 0.6638 - root_mean_squared_error: 1.1685
Score for fold 5: loss of 0.6638228893280029; root_mean_squared_error of 1.168514609336853
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.49131613969802856 - RMSE: 0.9932130575180054
------------------------------------------------------------------------
> Fold 2 - Loss: 0.40823644399642944 - RMSE: 0.9040364623069763
------------------------------------------------------------------------
> Fold 3 - Loss: 0.33294743299484253 - RMSE: 0.8161076903343201
------------------------------------------------------------------------
> Fold 4 - Loss: 0.7616763710975647 - RMSE: 1.2668042182922363
------------------------------------------------------------------------
> Fold 5 - Loss: 0.6638228893280029 - RMSE: 1.168514609336853
-----------------------------------

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▄▁▁▁▂▄▁▂▅▃▂▄▁▄▂▅▅▄▄▅▆▅▄▃▄▃▄▄▅▃▄▄▄▃▅▅▄▃
val_root_mean_squared_error,█▁▄▂▂▁▃▄▁▂▅▄▃▄▁▄▂▅▅▄▄▅▆▅▄▃▄▃▄▄▅▄▄▄▄▄▄▄▄▃
Mean Validation Loss (all folds),0.5316
Mean Validation RMSE (all folds),1.02974
best_epoch,28
best_val_loss,0.01725


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: yr3zdpkc with config:
wandb: 	batch_size: 128
wandb: 	dense1: 32
wandb: 	dense2: 32
wandb: 	dropout1: 0.07773994591298616
wandb: 	epochs: 500
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▁▂▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▂▂▂▂
val_root_mean_squared_error,█▂▂▂▁▁▁▂▂▁▂▂▂▂▂▂▁▂▂▂▂▁▂▂▂▁▂▁▁▂▁▁▁▂▁▁▂▂▂▂
best_epoch,29
best_val_loss,0.01816
epoch,150
loss,0.07929
root_mean_squared_error,0.41292
val_loss,0.44217


Hyperparameter optimization and experiment recording all took place within the Weights and Biases platform. The set of hyperparametrs resulting in the lowest loss (in terms of RMSE) is noted in the top row of the image below:

<img src="./wandb_rmse.png" width="800">

In [24]:
# Enter best params from sweep
best_params = {
    "dropout": 0.2,
    "dense1": 2048,
    "dense2": 2048,
    "epochs": 750,
    "batch_size": 256
}

# Build model using the best parameters
def make_predictions(best_params, dataset):
    
    model = Sequential()
    model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(best_params.get("dense1"), activation='linear'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(best_params.get("dense2"), activation='linear'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss=Huber(), metrics=[RootMeanSquaredError()])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(dataset, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [25]:
# Make predictions
test_predictions = make_predictions(best_params, X_test)
holdout_predictions = make_predictions(best_params, X_holdout)

# Undo the log transform
test_predictions = np.exp(test_predictions)
holdout_predictions = np.exp(holdout_predictions)

# Generating submission CSV
d = {"Id":X_test.index,"SalePrice":test_predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

submission["Id"] = submission["Id"] + 1461

submission.to_csv("submission_nn.csv",index=False)

In [27]:
test_predictions

array([[228658.5 ],
       [266291.47],
       [385278.97],
       ...,
       [324904.97],
       [207755.81],
       [433188.25]], dtype=float32)

In [28]:
holdout_predictions

array([[ 218479.58 ],
       [ 133389.95 ],
       [ 132196.88 ],
       [ 132154.52 ],
       [ 163463.19 ],
       [ 335370.78 ],
       [ 132196.88 ],
       [ 130181.1  ],
       [ 233474.55 ],
       [ 143973.17 ],
       [ 132196.88 ],
       [ 159368.34 ],
       [ 174977.47 ],
       [ 285044.88 ],
       [ 132196.88 ],
       [ 132196.88 ],
       [ 132196.88 ],
       [ 141738.36 ],
       [ 200083.   ],
       [ 132196.88 ],
       [ 132196.88 ],
       [ 231247.31 ],
       [ 132196.88 ],
       [ 141900.39 ],
       [ 132196.88 ],
       [ 177879.97 ],
       [ 141840.06 ],
       [ 176016.47 ],
       [ 221649.62 ],
       [ 240287.45 ],
       [ 377281.47 ],
       [ 132196.88 ],
       [ 351849.88 ],
       [ 278087.8  ],
       [ 199817.2  ],
       [ 235199.83 ],
       [ 147299.16 ],
       [ 191182.55 ],
       [ 570931.2  ],
       [ 132196.88 ],
       [ 491676.2  ],
       [ 146617.45 ],
       [ 301641.62 ],
       [ 264456.66 ],
       [ 212061.81 ],
       [ 1

## Blending Predictions

So far, I have generated predictions using a decision tree-based model and a neural net-based model.
First, I'll try taking the mean of predictions from both to see how it performs.

In [33]:
# Read CSVs
decision_tree_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
neural_net_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

# Merge CSVs on Id column
decision_tree_predictions["neural_net_predictions"] = neural_net_predictions["SalePrice"]

# Rename decision tree predictions column
decision_tree_predictions = decision_tree_predictions.rename(columns={"SalePrice":"decision_tree_predictions"})

# Add new column with mean
decision_tree_predictions["SalePrice"] = decision_tree_predictions[["neural_net_predictions","decision_tree_predictions"]].mean(axis=1)

# Drop other columns
decision_tree_predictions = decision_tree_predictions[["Id","SalePrice"]]

# Create mean prediction submission CSV
mean_predictions = decision_tree_predictions
mean_predictions.to_csv("submission_mean.csv",index=False)

Taking the mean of predictions did not beat my score from just using LightGBM in the previous post.

<img src="./mean_predictions_kaggle.png" width="600">

Next, I'll try defining a meta-model to best blend predictions from the two models. After developing each model, predictions were also made on a holdout dataset that was kept separate from the training and validation datasets, for the explicit purpose of training this meta-model. The model trained on this dataset was then used to blend predictions on the test dataset to be submitted to Kaggle.

In [34]:
# Append holdout set NN predictions to DT predictions
holdout_predictions_df["NN_predictions"] = holdout_predictions
holdout_predictions_df = holdout_predictions_df[["DT_predictions", "NN_predictions", "Actual_SalePrice"]]

# Reverse log transform on SalePrice
holdout_predictions_df["Actual_SalePrice"] = np.exp(holdout_predictions_df["Actual_SalePrice"])

In [35]:
holdout_predictions_df

,DT_predictions,NN_predictions,Actual_SalePrice
0,200565.22,218479.578125,208500.0
1,133316.58,133389.953125,129500.0
2,131912.38,132196.875000,132000.0
3,107866.20,132154.515625,90000.0
4,155335.47,163463.187500,159000.0
...,...,...,...
287,234057.98,286442.281250,271000.0
288,171889.88,201950.968750,192140.0
289,144124.02,142785.062500,143750.0
290,110798.66,132196.875000,64500.0


In [37]:
# Get predictions on test set
test_predictions_dt = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission_dt.csv")
test_predictions_nn = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission_nn.csv")

# Merge CSVs on Id column
test_predictions_dt["NN_predictions"] = test_predictions_nn["SalePrice"]

# Rename decision tree predictions column
test_predictions_dt = test_predictions_dt.rename(columns={"SalePrice":"DT_predictions"})

# Rename df
blended_predictions_df = test_predictions_dt

The meta-model was a simple grid search across different estimators, without attempting to optimize hyperparameters. Since the holdout dataset is very small, I used 10 folds in the cross-validation process.

In [38]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# Defining a custom loss function (Root Mean Squared Error)
rmse = make_scorer(mean_squared_error, squared=False)

X = holdout_predictions_df[["DT_predictions", "NN_predictions"]]
y = holdout_predictions_df["Actual_SalePrice"]

estimators = [
    {
        "clf": (LinearRegression(),)
    },
    {
        "clf": (Ridge(),)
    },
    {
        "clf": (xgb.XGBRegressor(),)
    },
    {
       "clf": (lgb.LGBMRegressor(),)
    }
]

pipe = Pipeline([("clf", LinearRegression())])

grid_search = GridSearchCV(pipe, estimators, cv=10, scoring=rmse)
grid_search.fit(X,y)

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the 

GridSearchCV(cv=10, estimator=Pipeline(steps=[('clf', LinearRegression())]),
             param_grid=[{'clf': (LinearRegression(),)}, {'clf': (Ridge(),)},
                         {'clf': (XGBRegressor(base_score=None, booster=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               enable_categorical=False,
                                               gamma=None, gpu_id=None,
                                               importance_type=None,
                                               interaction_constraints=None,...
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimator

In [39]:
# Printing the best estimator
grid_search.best_estimator_

Pipeline(steps=[('clf', LGBMRegressor())])

In [40]:
# Make predictions 
grid_search_blended_predictions = grid_search.predict(blended_predictions_df[["DT_predictions", "NN_predictions"]])

In [41]:
# Add predictions to a dataframe
grid_search_blended_predictions_df = blended_predictions_df
grid_search_blended_predictions_df["SalePrice"] = grid_search_blended_predictions
grid_search_blended_predictions_df = grid_search_blended_predictions_df.drop(columns=["DT_predictions","NN_predictions"])

# Create CSV with blended predictions
grid_search_blended_predictions_df.to_csv("submission_gridsearch_blended.csv",index=False)

In [42]:
grid_search_blended_predictions_df

,Id,SalePrice
0,1461,136766.464370
1,1462,186703.082381
2,1463,274587.653517
3,1464,280045.176750
4,1465,274587.653517
...,...,...
1454,2915,101520.160747
1455,2916,101520.160747
1456,2917,186667.541492
1457,2918,120444.947406


I had high hopes for a meta-model that blended predictions, but it performed worse than I expected.



## Improvements to subsequent models

Below I note some improvements to the models I would implement if I had more time.

- Remove outliers (using something like sklearn's IsolationForest)
- 